## 02_解析雨量_連続画像からOpenCVで動画作成

### 概要：

「01_解析雨量_連続画像出力」で連続出力した画像に、日付時間テキストを追加後、動画を作成するサンプルです。
- 使っているPython env にOpenCV をconda か pip でインストールしておく必要があります。

> conda install opencv  

or  

> pip install opencv-python  

## 連続画像から動画作成のメイン部分

### 変数定義（↓適宜修正してご利用下さい）

In [7]:
img_folder = r"ExportImg" # 01_解析雨量_連続画像出力で指定した画像を出力するフォルダー名
tmp_folder = r"temp" # 動画作成前に日付時間テキストを追加したpng画像を保存するテンポラリフォルダー名

# 時間テキストに関する部分
txt_org = (500,800) # 時間テキストを表示する左下の位置
ft_scale = 1.0 # 時間テキストのフォントスケール
ft_color = (0,0,0) # 時間テキストの色(r,g,b)で指定
ft_thicknass = 1 # 時間テキストの線の太さ
bl_use_jst = False # デフォルトは世界標準時のFalse。日本標準時へ変換した時間テキストにしたい場合はTrueに変更

# 動画関連
video_file_name = r"imgvideo.mp4" #r"imgvideo_jst.mp4" # 動画ファイル名（形式はmp4）
video_fps = 5 # フレームレート（fps）

### 前処理

In [8]:
import os
import arcpy
import cv2
import glob
import shutil
from datetime import datetime,timedelta,timezone

# 関数定義
def short_filename_to_utc_timestamp(short_filename):
    """
    短縮したファイル名から datetime型(utc)に変換
    例）
    Z__C_RJTD_20160103_2330_ANAL_grib2
    →
    20160103_2330
    →
    datetime型(utc) に変更
    """
    str_datetime = short_filename[10:23] #20160103_2330
    year = int(str_datetime[0:4])
    month = int(str_datetime[4:6])
    day = int(str_datetime[6:8])
    hour = int(str_datetime[9:11])
    minute = int(str_datetime[11:13])
    utc_timestamp = datetime(year=year, month=month, day=day, hour=hour, minute=minute, tzinfo=timezone.utc)
    return utc_timestamp

def utc_to_jst_timestamp(utc_timestamp):
    """
    datetime型(utc) からdatetime型(jst)  に変換
    """
    JST = timezone(timedelta(hours=+9))
    return utc_timestamp.astimezone(JST)

#サンプルケース
#shortname="Z__C_RJTD_20160103_2330_ANAL_grib2"
#utc_dt=short_filename_to_utc_timestamp(shortname)
#utc_dt.strftime('%Y-%m-%d %H:%M:%S')
#jst_dt=utc_to_jst_timestamp(utc_dt)
#jst_dt.strftime('%Y-%m-%d %H:%M:%S')

# 現在のプロジェクトからmap と activeview を取得
aprx = arcpy.mp.ArcGISProject("current")
m = aprx.activeMap
mv = aprx.activeView
# 画像を出力してあるフォルダーを、現在のプロジェクト直下に指定
export_folder = os.path.join(aprx.homeFolder,img_folder)
temp_folder = os.path.join(export_folder, tmp_folder)
# テンポラリフォルダーが存在しない場合は新規に作成
if os.path.exists(temp_folder) == False:
    os.mkdir(temp_folder)
# ビデオファイルが存在する場合は削除
video_file = os.path.join(export_folder, video_file_name)
if os.path.exists(video_file):
    os.remove(video_file)

### 連続画像のファイル一覧を取得し、ファイル名をもとに日付時間テキストを追加した画像をテンポラリフォルダーに保存

OpenCV -> putText()  
 - img : Image.
 - text : Text string to be drawn.
 - org : Bottom-left corner of the text string in the image.
 - fontFace : Font type, see [HersheyFonts](https://docs.opencv.org/4.x/d6/d6e/group__imgproc__draw.html#ga0f9314ea6e35f99bb23f29567fc16e11) .
 - fonrScale : Font scale factor that is multiplied by the font-specific base size.
 - color : Text color.
 - thicknass : 	Thickness of the lines used to draw a text.
 - lineType : Line type. See [LineTypes](https://docs.opencv.org/4.x/d6/d6e/group__imgproc__draw.html#gaf076ef45de481ac96e0ab3dc2c29a777) .
 - bottomLeftOrigin : When true, the image data origin is at the bottom-left corner. Otherwise, it is at the top-left corner.
  
https://docs.opencv.org/4.x/d6/d6e/group__imgproc__draw.html#ga5126f47f883d730f633d74f07456c576

In [9]:
pngfiles = glob.glob(export_folder + "/*.png") # 画像の一覧を取得
for filename in pngfiles:
    img = cv2.imread(filename) # ファイルをイメージに読込み
    base_filename = os.path.basename(filename)
    out_dt = short_filename_to_utc_timestamp(base_filename) #ファイル名から世界標準時のTimeStampへ変換
    if bl_use_jst: #日本標準時の場合は変換
        out_dt =  utc_to_jst_timestamp(out_dt)
    cv2.putText(img,
                text=out_dt.strftime('%Y-%m-%d %H:%M:%S'),
                org=txt_org,
                fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=ft_scale,
                color=ft_color,
                thickness=ft_thicknass,
                lineType=cv2.LINE_4 )
    new_filename = os.path.join(temp_folder, base_filename)
    cv2.imwrite(new_filename, img) #テンポラリフォルダーへ保存

### テンポラリフォルダーから画像一覧を取得しソート後、arrayに格納

In [10]:
#フォルダーから*.pngファイルの一覧を取得してソートします
tm_pngfiles = glob.glob(temp_folder + "/*.png")
tm_pngfiles = sorted(tm_pngfiles)

#arrayに画像ファイルを格納
img_array = []
for tm_pngfile in tm_pngfiles:
    img = cv2.imread(tm_pngfile)
    height, width, layers = img.shape
    size = (width, height)
    img_array.append(img)

### mp4の動画を作成します

OpenCV -> VideoWriter()
 - filename : Name of the output video file.
 - fourcc : 4-character code of codec used to compress the frames.
 - fps : Framerate of the created video stream.
 - frameSize : Size of the video frames.
 - isColor : If it is not zero, the encoder will expect and encode color frames, otherwise it will work with grayscale frames

https://docs.opencv.org/3.4/dd/d9e/classcv_1_1VideoWriter.html#ad59c61d8881ba2b2da22cff5487465b5

In [11]:
# mp4のコーデックを指定してvideoWriterのインスタンス作成し、imgを追加
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(filename=video_file, fourcc=fourcc, fps=video_fps, frameSize=size)
for img in img_array:
    video.write(img)
cv2.destroyAllWindows()
video.release()
del img_array

### テンポラリフォルダーの後始末

In [12]:
# 中身のファイルごとテンポラリフォルダーを削除
shutil.rmtree(temp_folder)